<a href="https://colab.research.google.com/github/Lucas-Siade/Data-Science/blob/main/Projeto%20Final/Modelo%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
import random

## **Funções**

### **Carregar os Dados**

In [2]:
def carregarDados(path):
    dataset = pd.read_csv(path, sep="\t")

    dataset = dataset.drop(columns=["<SPREAD>"])
    dataset = dataset.rename(columns={
        "<DATE>":"DATE",
        "<TIME>":"TIME",
        "<OPEN>":"OPEN",
        "<CLOSE>":"CLOSE",
        "<HIGH>":"HIGH",
        "<LOW>":"LOW",
        "<TICKVOL>":"TICKVOL",
        "<VOL>":"VOL"
    })

    dataset["TIMESTAMP"] = pd.to_datetime(dataset["DATE"] + " " + dataset["TIME"], format="%Y.%m.%d %H:%M:%S")
    dataset.drop(columns=["DATE", "TIME"], inplace=True)
    dataset = dataset[["TIMESTAMP", "OPEN", "HIGH", "LOW", "CLOSE", "TICKVOL", "VOL"]]

    return dataset

### **Adicionar dias anteriores**

In [3]:
def adicionarAnteriores(dataset, dias):
    datasetCopy = dataset.copy()

    for i in range(1, dias + 1):
        for coluna in ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL']:
            datasetCopy[f'{coluna}_{i - 1}'] = datasetCopy[coluna].shift(i)

    return datasetCopy

### **Remover valores do dia atual**

In [4]:
def removerValores(dataset):
    colunas = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL']

    return dataset.drop(columns=[col for col in colunas])

## **Modelos**

### **Tendência**

In [5]:
import xgboost as XGB

path = "/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/Dados.csv"
dataset = carregarDados(path)
dias = 2

dataset = dataset.copy()
dataset = adicionarAnteriores(dataset, dias)
dataset = dataset.iloc[dias:].reset_index(drop=True)
dataset = removerValores(dataset)

modelTend = joblib.load('/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelTendFinal.joblib')
scalerTend = joblib.load("/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/scalerTendFinal.save")

colunasX = [col for col in dataset.columns if any(p in col for p in ['OPEN_', 'HIGH_', 'LOW_', 'CLOSE_', 'TICKVOL_', 'VOL_'])]

dadosX_scaled = scalerTend.transform(dataset[colunasX])
DMatrix = XGB.DMatrix(dadosX_scaled)

probabilidades = modelTend.predict(DMatrix)

dataset["PROBABILIDADE"] = probabilidades * 100
dataset["PROBABILIDADE"] = dataset["PROBABILIDADE"].round(2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [6]:
display(dataset.head())

,TIMESTAMP,OPEN_0,HIGH_0,LOW_0,CLOSE_0,TICKVOL_0,VOL_0,OPEN_1,HIGH_1,LOW_1,CLOSE_1,TICKVOL_1,VOL_1,PROBABILIDADE
0,2020-10-15 11:51:00,98570.0,98625.0,98570.0,98600.0,5483.0,19111.0,98565.0,98575.0,98530.0,98575.0,2373.0,9001.0,25.260000
1,2020-10-15 11:52:00,98595.0,98600.0,98535.0,98580.0,4572.0,17663.0,98570.0,98625.0,98570.0,98600.0,5483.0,19111.0,90.470001
2,2020-10-15 11:53:00,98575.0,98620.0,98535.0,98615.0,3536.0,13143.0,98595.0,98600.0,98535.0,98580.0,4572.0,17663.0,49.130001
3,2020-10-15 11:54:00,98620.0,98625.0,98575.0,98610.0,3729.0,13702.0,98575.0,98620.0,98535.0,98615.0,3536.0,13143.0,65.620003
4,2020-10-15 11:55:00,98610.0,98655.0,98590.0,98655.0,5040.0,17128.0,98620.0,98625.0,98575.0,98610.0,3729.0,13702.0,78.139999


### **Prever Valor**

In [7]:
modelPred = joblib.load('/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelPredFinal.joblib')
scalerX = joblib.load("/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/scalerXFinal.joblib")
scalerY = joblib.load("/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/scalerYFinal.joblib")

X = dataset[colunasX].values
X = np.concatenate([X, dataset[['PROBABILIDADE']].values], axis=1)
X_scaled = scalerX.transform(X)

Y_Pred_scaled = modelPred.predict(X_scaled)
Y_Pred = scalerY.inverse_transform(Y_Pred_scaled)

dataset['CLOSE_PREDITO'] = Y_Pred.flatten()

10274/10274 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step


In [8]:
display(dataset.head())

,TIMESTAMP,OPEN_0,HIGH_0,LOW_0,CLOSE_0,TICKVOL_0,VOL_0,OPEN_1,HIGH_1,LOW_1,CLOSE_1,TICKVOL_1,VOL_1,PROBABILIDADE,CLOSE_PREDITO
0,2020-10-15 11:51:00,98570.0,98625.0,98570.0,98600.0,5483.0,19111.0,98565.0,98575.0,98530.0,98575.0,2373.0,9001.0,25.260000,98604.820312
1,2020-10-15 11:52:00,98595.0,98600.0,98535.0,98580.0,4572.0,17663.0,98570.0,98625.0,98570.0,98600.0,5483.0,19111.0,90.470001,98576.679688
2,2020-10-15 11:53:00,98575.0,98620.0,98535.0,98615.0,3536.0,13143.0,98595.0,98600.0,98535.0,98580.0,4572.0,17663.0,49.130001,98603.703125
3,2020-10-15 11:54:00,98620.0,98625.0,98575.0,98610.0,3729.0,13702.0,98575.0,98620.0,98535.0,98615.0,3536.0,13143.0,65.620003,98609.750000
4,2020-10-15 11:55:00,98610.0,98655.0,98590.0,98655.0,5040.0,17128.0,98620.0,98625.0,98575.0,98610.0,3729.0,13702.0,78.139999,98646.398438


## **Tabela Original**

In [9]:
datasetOriginal = carregarDados(path)
datasetOriginal = datasetOriginal.iloc[2:].reset_index(drop=True)

display(datasetOriginal.head())

,TIMESTAMP,OPEN,HIGH,LOW,CLOSE,TICKVOL,VOL
0,2020-10-15 11:51:00,98595,98600,98535,98580,4572,17663
1,2020-10-15 11:52:00,98575,98620,98535,98615,3536,13143
2,2020-10-15 11:53:00,98620,98625,98575,98610,3729,13702
3,2020-10-15 11:54:00,98610,98655,98590,98655,5040,17128
4,2020-10-15 11:55:00,98650,98665,98610,98615,4021,13669


# **Treino do Modelo**

In [10]:
#DIVISÃO DOS DADOS
tamanhoTreino = len(dataset) // 5
datasetTreino = dataset.iloc[:tamanhoTreino]
datasetOriginalTreino = datasetOriginal.iloc[:tamanhoTreino]

colunasNumericas = dataset.drop(columns=['TIMESTAMP']).columns
maxValor = dataset[colunasNumericas].max().max()

def get_state(row):
    return torch.tensor([row[col] / maxValor for col in colunasNumericas], dtype=torch.float32)

#MODELO PPO
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(PolicyNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        return self.model(x)

#PARÂMETROS
policy_net = PolicyNetwork(input_dim=len(colunasNumericas))
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)

gamma = 0.95
episodes = 15

#TREINO DO MODELO
for ep in range(episodes):
    comprado = False
    preco_compra = 0
    log_probs = []
    rewards = []
    total_reward = 0

    for i in range(len(datasetTreino) - 1):
        row = datasetTreino.iloc[i]
        row_original = datasetOriginalTreino.iloc[i]

        state = get_state(row)

        logits = policy_net(state)
        probs = torch.softmax(logits, dim=0)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        if not comprado:
            acao = ['COMPRAR', 'NADA'][action.item()]
        else:
            acao = ['VENDER', 'NADA'][action.item()]

        recompensa = 0

        if acao == 'COMPRAR' and not comprado:
            comprado = True
            preco_compra = row_original['OPEN']

        elif acao == 'VENDER' and comprado:
            preco_venda = row['CLOSE_PREDITO']
            recompensa = preco_venda - preco_compra
            comprado = False
            preco_compra = 0

        elif acao == 'NADA':
            recompensa = 1

        log_probs.append(log_prob)
        rewards.append(recompensa)
        total_reward += recompensa

    returns = []
    R = 0
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + 1e-9)

    loss = sum(-lp * R for lp, R in zip(log_probs, returns))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Episódio {ep+1} | Recompensa Total: {total_reward:.2f}")

Episódio 1 | Recompensa Total: 309194.72
Episódio 2 | Recompensa Total: 316727.01
Episódio 3 | Recompensa Total: 340805.96
Episódio 4 | Recompensa Total: 324018.59
Episódio 5 | Recompensa Total: 330691.91
Episódio 6 | Recompensa Total: 326878.16
Episódio 7 | Recompensa Total: 332302.02
Episódio 8 | Recompensa Total: 350605.91
Episódio 9 | Recompensa Total: 335214.34
Episódio 10 | Recompensa Total: 355022.41
Episódio 11 | Recompensa Total: 352121.02
Episódio 12 | Recompensa Total: 357332.19
Episódio 13 | Recompensa Total: 363083.36
Episódio 14 | Recompensa Total: 361437.10
Episódio 15 | Recompensa Total: 367024.09


In [11]:
#PARÂMETROS
episodes = 25
partes = 5
episodios_por_parte = episodes // partes
tamanho_parte = len(dataset) // partes
gamma = 0.95

colunasNumericas = dataset.drop(columns=['TIMESTAMP']).columns
maxValor = dataset[colunasNumericas].max().max()

def get_state(row):
    return torch.tensor([row[col] / maxValor for col in colunasNumericas], dtype=torch.float32)

#MODELO PPO
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(PolicyNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )
    def forward(self, x):
        return self.model(x)

policy_net = PolicyNetwork(input_dim=len(colunasNumericas))
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)

melhoresOrdens = []

#TREINO DO MODELO
for parte in range(partes):
    inicio = parte * tamanho_parte
    fim = (parte + 1) * tamanho_parte

    datasetTreino = dataset.iloc[inicio:fim].reset_index(drop=True)
    datasetOriginalTreino = datasetOriginal.iloc[inicio:fim].reset_index(drop=True)

    print(f"\n=== Treinando com dados da Parte {parte+1}/{partes} (linhas {inicio} a {fim}) ===")

    melhor_lucro = float('-inf')
    melhor_historico = []

    for ep in range(episodios_por_parte):
        comprado = False
        preco_compra = 0
        log_probs = []
        rewards = []
        total_reward = 0
        historicoCompleto = []

        for i in range(len(datasetTreino) - 1):
            row = datasetTreino.iloc[i]
            row_original = datasetOriginalTreino.iloc[i]

            state = get_state(row)

            logits = policy_net(state)
            probs = torch.softmax(logits, dim=0)
            dist = torch.distributions.Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)

            if not comprado:
                acoesPossiveis = ['COMPRAR', 'NADA']
            else:
                acoesPossiveis = ['VENDER', 'NADA']

            acao = acoesPossiveis[action.item()]
            recompensa = 0
            info = {}

            if acao == 'COMPRAR' and not comprado:
                comprado = True
                preco_compra = row_original['OPEN']
                info = {'ação': 'COMPRAR', 'preço_compra': preco_compra}

            elif acao == 'VENDER' and comprado:
                preco_venda = row['CLOSE_PREDITO']
                recompensa = preco_venda - preco_compra
                info = {
                    'ação': 'VENDER',
                    'preço_compra': preco_compra,
                    'preço_venda': preco_venda,
                    'lucro': recompensa
                }
                comprado = False
                preco_compra = 0

            elif acao == 'NADA':
                recompensa = 1
                info = {'ação': 'NADA'}

            log_probs.append(log_prob)
            rewards.append(recompensa)
            total_reward += recompensa
            historicoCompleto.append((row['TIMESTAMP'], info))

        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        loss = sum(-lp * R for lp, R in zip(log_probs, returns))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lucroTotal = sum(info['lucro'] for _, info in historicoCompleto if info['ação'] == 'VENDER')

        print(f"Parte {parte+1} | Episódio {ep+1}/{episodios_por_parte} | Lucro Total: {lucroTotal:.2f}")

        if lucroTotal > melhor_lucro:
            melhor_lucro = lucroTotal
            melhor_historico = historicoCompleto

    melhoresOrdens.extend(melhor_historico)

melhor_df = pd.DataFrame([
    {
        'TIMESTAMP': tempo,
        'AÇÃO': info['ação'],
        'PREÇO_COMPRA': info.get('preço_compra', None),
        'PREÇO_VENDA': info.get('preço_venda', None),
        'LUCRO': info.get('lucro', None)
    }
    for tempo, info in melhoresOrdens
])

lucro_final = melhor_df['LUCRO'].dropna().sum()
print(f"\nLucro total acumulado dos MELHORES episódios: {lucro_final:.2f}")


=== Treinando com dados da Parte 1/5 (linhas 0 a 65752) ===
Parte 1 | Episódio 1/5 | Lucro Total: 318997.61
Parte 1 | Episódio 2/5 | Lucro Total: 331053.50
Parte 1 | Episódio 3/5 | Lucro Total: 340939.31
Parte 1 | Episódio 4/5 | Lucro Total: 335430.09
Parte 1 | Episódio 5/5 | Lucro Total: 347518.88

=== Treinando com dados da Parte 2/5 (linhas 65752 a 131504) ===
Parte 2 | Episódio 1/5 | Lucro Total: 480684.01
Parte 2 | Episódio 2/5 | Lucro Total: 484582.91
Parte 2 | Episódio 3/5 | Lucro Total: 492156.58
Parte 2 | Episódio 4/5 | Lucro Total: 482735.93
Parte 2 | Episódio 5/5 | Lucro Total: 501881.71

=== Treinando com dados da Parte 3/5 (linhas 131504 a 197256) ===
Parte 3 | Episódio 1/5 | Lucro Total: 323591.61
Parte 3 | Episódio 2/5 | Lucro Total: 343818.78
Parte 3 | Episódio 3/5 | Lucro Total: 345679.34
Parte 3 | Episódio 4/5 | Lucro Total: 341994.52
Parte 3 | Episódio 5/5 | Lucro Total: 357876.93

=== Treinando com dados da Parte 4/5 (linhas 197256 a 263008) ===
Parte 4 | Episódio 

In [12]:
dadosTabela = []
tempo_compra = None

for tempo, info in melhoresOrdens:
    ordem = info.get('ação', 'NADA').upper()
    lucro = info['lucro'] if ordem == 'VENDER' else None
    duracao = None

    if ordem == 'COMPRAR':
        tempo_compra = tempo
    elif ordem == 'VENDER' and tempo_compra is not None:
        try:
            duracao = pd.to_datetime(tempo) - pd.to_datetime(tempo_compra)
            duracao = duracao.total_seconds() / 60
        except:
            duracao = None
        tempo_compra = None

    dadosTabela.append({
        'TIMESTAMP': tempo,
        'ORDEM': ordem,
        'LUCRO': lucro,
        'DURACAO (MINUTOS)': duracao
    })

tabela = pd.DataFrame(dadosTabela)
display(tabela.tail())

tabela.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/ResultadoFinal.csv",
    index=False,
    sep=';',
    decimal=','
)

,TIMESTAMP,ORDEM,LUCRO,DURACAO (MINUTOS)
328750,2023-04-19 17:49:00,NADA,NaN,NaN
328751,2023-04-19 17:50:00,VENDER,-15.390625,2.0
328752,2023-04-19 17:51:00,COMPRAR,NaN,NaN
328753,2023-04-19 17:52:00,VENDER,7.695312,1.0
328754,2023-04-19 17:53:00,NADA,NaN,NaN
